In [1]:
import shutil
import urllib.request as request
from contextlib import closing
import zipfile
import tarfile
import os
import glob


tars = {
    "all": "https://tgftp.nws.noaa.gov/SL.us008001/DF.sha/DC.cap/DS.WWA/current_all.tar.gz",
    "hazards": "https://tgftp.nws.noaa.gov/SL.us008001/DF.sha/DC.cap/DS.WWA/current_hazards.tar.gz",
    "warnings": "https://tgftp.nws.noaa.gov/SL.us008001/DF.sha/DC.cap/DS.WWA/current_warnings.tar.gz",
}


def clear_folder_contents(folder):
  '''
  Clears all contents of folder specified in the argument
  '''
  for file in os.listdir(folder):
    file_path = os.path.join(folder, file)
    try:
      if os.path.isfile(file_path):
        os.unlink(file_path)
      elif os.path.isdir(file_path): shutil.rmtree(file_path)
    except Exception as e:
      print(e)


def download_tar_file(file_url: str):
    '''
    Downloads a tar file and extracts it in a target directory of choice
    '''
    file = file_url.split('/')[-1]

    with closing(request.urlopen(file_url)) as r:
        with open(file, 'wb') as f:
          shutil.copyfileobj(r, f)
    
    tar = tarfile.open(file, mode='r')
    tar.extractall('data')
  
  
# Clear directories before getting data
clear_folder_contents('data')
    
# Get the hazard tar files
for url in tars:
    download_tar_file(tars[url])
    
# Move tars
if not os.path.exists('data/tars'):
    os.makedirs('data/tars')
    
for tar_file in glob.glob(f"{os.getcwd()}/*.tar.gz"):
  head, tail = os.path.split(tar_file)
  shutil.move(tar_file, f"data/tars/{tail}")